# Select and Train a Model

- github colab : https://homl.info/colab3

In [18]:
# 지금까지 한 것의 총합 : Pipeline을 사용해서 처리

import matplotlib
import matplotlib.pyplot
import numpy
import pathlib
import pandas
import sklearn
import sklearn.base
import sklearn.compose
import sklearn.cluster
import sklearn.ensemble
import sklearn.impute
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics.pairwise
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.tree
import sklearn.utils.validation
import tarfile
import urllib

def ch2_load_housing_data():
    tarball_path = pathlib.Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        pathlib.Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pandas.read_csv(pathlib.Path("datasets/housing/housing.csv"))

def matplotlib_to_imagefile(output_dir, filename, imgext="png", tight_layout=True, resolution=300):
    path = output_dir / f"{filename}.{imgext}"
    if tight_layout:
        matplotlib.pyplot.tight_layout()
    matplotlib.pyplot.savefig(path, format=imgext, dpi=resolution)
    
def stratified_sampling_income_category(input_dataframe):
    input_dataframe = input_dataframe.copy()
    input_dataframe["income_cat"] = pandas.cut(input_dataframe["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., numpy.inf],
                               labels=[1, 2, 3, 4, 5])
    s_train, s_test = sklearn.model_selection.train_test_split(input_dataframe, test_size = 0.2, stratify = input_dataframe['income_cat'], random_state = 42)
    s_train.drop('income_cat', axis=1, inplace=True)
    s_test.drop('income_cat', axis=1, inplace=True)
    
    return s_train, s_test
    
# 저장할 디렉토리 설정
output_dir = pathlib.Path() / "images" / "end_to_end_project"
output_dir.mkdir(parents=True, exist_ok=True)
print(f'output_dir : {output_dir}')

input_dataframe = ch2_load_housing_data()
train, test = stratified_sampling_income_category(input_dataframe)

# train set에서 label과 predictor를 분리한다
label = train['median_house_value'].copy()
predictor = train.drop('median_house_value', axis = 1)

# Proprocessing용 pipeline을 준비
num_pipeline = sklearn.pipeline.make_pipeline(
    sklearn.impute.SimpleImputer(strategy='median'), 
    sklearn.preprocessing.StandardScaler())

cat_pipeline = sklearn.pipeline.make_pipeline(
    sklearn.impute.SimpleImputer(strategy='most_frequent'),
    sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore'))

def column_ratio(X):
    return X[:,[0]] / X[:,[1]]

#
# 여기에서 왜 length하나짜리 list를 반환하는지 꽤 헷갈렸다
# 아래 실행한걸 보면 알수 있듯이 ColumnTransformer를 쓰면 feature의 앞부분에 일단 
# name이 알아서 앞에 붙는다
# 그래서 name__ratio 이런식으로 나오게 하는거다
#
def ratio_name(function_transformer, feature_names_in):
    return ['ratio']

def ratio_pipeline():
    return sklearn.pipeline.make_pipeline(
        sklearn.impute.SimpleImputer(strategy='median'),
        sklearn.preprocessing.FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        # sklearn.preprocessing.FunctionTransformer(numpy.log, feature_names_out='one-to-one'),
        sklearn.preprocessing.StandardScaler(),
    )

class ClusterSimilarity(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = sklearn.cluster.KMeans(self.n_clusters, n_init='auto', random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        return sklearn.metrics.pairwise.rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

#
# numpy.log는 AxB matrix를 받아서 AxB matrix를 반환한다
# 즉 아래에서 5 column 짜리 matrix를 넣으면 5 column 짜리 matrix를 반환한다
# 그래서 각각 이름을 붙이려면 length가 5인 array를 리턴하는 함수(위의 ratio_name 스타일인데 5개짜리 array를 리턴하는)
# 를 넣어주던지 아니면 그냥 이름을 그대로 쓰는 one-to-one을 쓰던지 하면 된다
#
log_pipeline = sklearn.pipeline.make_pipeline(
    sklearn.impute.SimpleImputer(strategy='median'),
    sklearn.preprocessing.FunctionTransformer(numpy.log, feature_names_out='one-to-one'),
    sklearn.preprocessing.StandardScaler(),
)

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1.0, random_state=42)

default_num_pipeline = sklearn.pipeline.make_pipeline(
    sklearn.impute.SimpleImputer(strategy='median'),
    sklearn.preprocessing.StandardScaler()
)

preprocessing = sklearn.compose.ColumnTransformer([
    # column_ratio는 Nx2 matrix를 받아서 Nx1 matrix를 반환한다
    ('bedrooms', ratio_pipeline(), ['total_bedrooms', 'total_rooms']),
    ('rooms_per_house', ratio_pipeline(), ['total_rooms', 'households']),
    ('people_per_house', ratio_pipeline(), ['population', 'households']),
    # numpy.log는 AxB matrix를 받아서 AxB matrix를 반환한다
    # 즉 아래에서 5 column 짜리 matrix를 넣으면 5 column 짜리 matrix를 반환한다
    ('log', log_pipeline, ['total_bedrooms', 'total_rooms', 'population','households','median_income']),
    ('geo', cluster_simil, ['latitude','longitude']),
    ('cat', cat_pipeline, sklearn.compose.make_column_selector(dtype_include=object)),
],
    remainder=default_num_pipeline) # one column remaining : housing_median_age

predictor_prepared = preprocessing.fit_transform(predictor)

print(predictor_prepared.shape)

preprocessing.get_feature_names_out()

output_dir : images\end_to_end_project
(16512, 24)


array(['bedrooms__ratio', 'rooms_per_house__ratio',
       'people_per_house__ratio', 'log__total_bedrooms',
       'log__total_rooms', 'log__population', 'log__households',
       'log__median_income', 'geo__Cluster 0 similarity',
       'geo__Cluster 1 similarity', 'geo__Cluster 2 similarity',
       'geo__Cluster 3 similarity', 'geo__Cluster 4 similarity',
       'geo__Cluster 5 similarity', 'geo__Cluster 6 similarity',
       'geo__Cluster 7 similarity', 'geo__Cluster 8 similarity',
       'geo__Cluster 9 similarity', 'cat__ocean_proximity_<1H OCEAN',
       'cat__ocean_proximity_INLAND', 'cat__ocean_proximity_ISLAND',
       'cat__ocean_proximity_NEAR BAY', 'cat__ocean_proximity_NEAR OCEAN',
       'remainder__housing_median_age'], dtype=object)

## Train and Evaluate on the Training Set

### Linear regression 예제

In [13]:
# preprocessing이 data를 preprocessing해 주는 전단계 pipeline인거고
# 아래 코드는 그걸 거친 data를 가지고 linear regression을 수행하는 코드다

lin_reg = sklearn.pipeline.make_pipeline(preprocessing, sklearn.linear_model.LinearRegression())
lin_reg.fit(predictor, label)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x000...
                                                   'median_income']),
                                                 ('geo',
                                                  ClusterSimilarity(random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000024AC521A910>)])),
                ('linearregression', LinearRegression())])

In [14]:
lin_prediction = lin_reg.predict(predictor)
print(lin_prediction[:5].round(-2))
print(label.iloc[:5].values)

[248700. 372800. 130800.  93600. 326300.]
[458300. 483800. 101700.  96100. 361800.]


In [15]:
# squared=True면 MSE, squared=False면 RMSE
lin_rmse = sklearn.metrics.mean_squared_error(label, lin_prediction, squared=False)
print(lin_rmse) # RMSE가 높게 나온다. 즉 underfitting이다

68831.98440583353


### Decision tree regression 예제
- 이건 어떻게 fit이 되는건지 상상이 안되네...

In [16]:
tree_reg = sklearn.pipeline.make_pipeline(preprocessing, sklearn.tree.DecisionTreeRegressor())
tree_reg.fit(predictor, label)
tree_prediction = tree_reg.predict(predictor)
tree_rmse = sklearn.metrics.mean_squared_error(label, tree_prediction, squared=False)
print(tree_rmse) # RMSE 가 0.0이 나온다. 즉 overfitting이다. 너무 낮게 나와도 문제, 너무 높게 나와도 문제...

0.0


- 여기쯤에서 Test Set을 사용하면 되지 않을까? 라는 생각이 들지만 책에서는 Test set은 Model을 Launch하는 시점까지는 건드리지 말라고 한다
- 그래서 Validation Set이라는게 필요하다. 즉 Test set은 내가 모델을 정말 제대로 잘 만들었다는 자신감, Launch해도 되겠다는 확신을 가지기 전까지는 정말로 건드리면 안되는 거고 Validation Set은 아직 model에 대한 연구/검증 단계에서 predictor와 구분되는 검증(hence, 'validation')용 set으로 이해해야 하는 것 같다
- 매우매우 편리하게도 Scikit-Learn에는 k_-fold cross validation 이라는게 있다
    - 이름에서 알수 있듯이 k개의 non-overlapping subset, 즉 fold를 생성한 다음에 k-1개로 train -> k개로 validate하는 기능이다

In [17]:
# Scikit-Learn의 cross-validation은 utility function (높을수록 좋음) 으로 학습하도록 되어 있다
# RMSE는 낮을수록 좋은 cost function이므로 neg_root_mean_squared_error를 사용하는 것이다
# 그래서 마지막에 -를 붙여서 RMSE로 바꾸는게 들어가 있는 것

tree_rmses = -sklearn.model_selection.cross_val_score(tree_reg, predictor, label, cv=10, scoring='neg_root_mean_squared_error')
pandas.Series(tree_rmses).describe() # 결과를 보면 Linear Regression과 큰 차이가 안난다...

count       10.000000
mean     66364.396384
std       1618.239800
min      64002.404346
25%      65516.226358
50%      66331.750091
75%      67403.677879
max      69099.169042
dtype: float64

### Random forest regression 예제
- Random subset of feature를 이용해서 여러 Decision tree를 만들고 거기서 나온 prediction을 average하는 거라고 한다
- 뭐 Chapter 7에서 제대로 다시 볼듯...

In [23]:
forest_reg = sklearn.pipeline.make_pipeline(preprocessing, sklearn.ensemble.RandomForestRegressor())

# Single-thread는 꽤 오래 걸림. i9-12900H (laptop)에서 4분 정도 걸림
# forest_rmses = -sklearn.model_selection.cross_val_score(forest_reg, predictor, label, cv=10, scoring='neg_root_mean_squared_error')

# Multi-thread 꼭 써야 하는 듯. i9-12900H (laptop)에서 30초 정도 걸림. n_jobs=-1 하면 모든 코어를 다 쓴다
forest_rmses = -sklearn.model_selection.cross_val_score(forest_reg, predictor, label, n_jobs=-1, cv=10, scoring='neg_root_mean_squared_error')


In [24]:
# 이거 결과가 확실히 다른 것들보다는 낫다는 걸 알 수 있음. 그래도 좋은 결과라고 하기는 애매하지만...
pandas.Series(forest_rmses).describe()

count       10.000000
mean     47081.400513
std       1113.536125
min      45451.849753
25%      46355.554032
50%      47225.980110
75%      47792.465098
max      49010.391436
dtype: float64